In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp rics

# RIC's

In [ ]:
#export
import numpy as np
import qutip as qt
from scipy.stats import ortho_group

from qbism.povm import *
from qbism.random import *
from qbism.kraus import *

One can also consider POVM's on real valued Hilbert spaces. RIC-POVM's (real informationally complete POVM's) will have $\frac{d(d+1)}{2}$ elements (unlike the complex case, where they would be $d^2$ elements).

SIC-POVM's in real Hilbert spaces correspond to sets of *real* equiangular lines, and unlike in the complex case, they can be proved *not* to exist in certain dimensions.

For purposes of testing out RIC's, let's define some useful functions:

In [ ]:
#export
def real_rand_ket(d):
    r"""
    Generates a random ket in real Hilbert space of dimension $d$.
    """
    return qt.Qobj(np.random.randn(d)).unit()

In [ ]:
#export
def real_rand_dm(d):
    r"""
    Generates a random density matrix for a real Hilbert space of dimension $d$.
    """
    return qt.Qobj(qt.rand_dm(d).full().real)

In [ ]:
#export
def rand_symmetric(d):
    r"""
    Generates a random $d \times d$ symmetric matrix. These matrices correspond to observables in real quantum mechanics, being the real analogue of Hermitian matrices: $\hat{S} = \hat{S}^{T}$.
    """
    M = qt.Qobj(np.random.randn(d,d))
    return M*M.trans() + M.trans()*M

In [ ]:
#export
def rand_orthogonal(d):
    r"""
    Generates a random $d \times d$ orthogonal matrix. These matrices correspond to time evolution in real quantum mechanics, being the real analogue of unitary matrices: $\hat{S}\hat{S}^{T} = \hat{I}$.
    """
    return qt.Qobj(ortho_group.rvs(d))

Let's generate a random RIC and check that it behaves like the more usual complex IC-POVM's we're used to. First, let's check that we can go back and forth between density matrices and probabilities:

In [ ]:
d = 3
povm = random_haar_povm(d, real=True)
phi = povm_phi(povm)
rho = real_rand_dm(d)
p = dm_probs(rho, povm)
assert np.allclose(rho, probs_dm(p, povm))

Then let's compare classical and quantum probabilities for some observable represented by a symmetric matrix:

In [ ]:
S = rand_symmetric(d)
vn = [v*v.dag() for v in S.eigenstates()[1]]

R = conditional_probs(vn, povm)
classical_probs = R @ p
quantum_probs = R @ phi @ p

post_povm_rho = sum([(e*rho).tr()*(e/e.tr()) for e in povm])
assert np.allclose(classical_probs, [(v*post_povm_rho).tr() for v in vn])
assert np.allclose(quantum_probs, [(v*rho).tr() for v in vn])

And finally, let's check out time evolution under an othogonal matrix:

In [ ]:
O = rand_orthogonal(d)
assert np.allclose(dm_probs(O*rho*O.trans(), povm), povm_map([O], povm) @ phi @ p)

As an example, let's consider the Petersen RIC in $d=4$ based on the [Petersen Graph](https://en.wikipedia.org/wiki/Petersen_graph) and the [Rectified 5-cell](http://eusebeia.dyndns.org/4d/rect5cell).


In [ ]:
#export
def petersen_povm():
    petersen_vertices = ["u1", "u2", "u3", "u4", "u5", "v1", "v2", "v3", "v4", "v5"]
    petersen_graph = \
        {"u1": ["v1", "u2", "u5"],
        "u2": ["u1", "v2", "u3"],
        "u3": ["u2", "v3", "u4"],
        "u4": ["u3", "v4", "u5"],
        "u5": ["u4", "v5", "u1"],
        "v1": ["u1", "v4", "v3"],
        "v2": ["u2", "v4", "v5"],
        "v3": ["v5", "v1", "u3"],
        "v4": ["u4", "v1", "v2"],
        "v5": ["u5", "v3", "v2"]}
    petersen_gram = np.array([[1 if a == b else (\
                               -2/3 if b in petersen_graph[a] else \
                               1/6) for b in petersen_vertices]\
                                        for a in petersen_vertices]) 
    U, D, V = np.linalg.svd(petersen_gram)
    petersen_states = [qt.Qobj(state) for state in V[:4].T @ np.sqrt(np.diag(D[:4]))]
    return [(2/5)*v*v.dag() for v in petersen_states]

petersen = petersen_povm()
assert np.allclose(sum(petersen), qt.identity(4))

rho = real_rand_dm(4)
assert np.allclose(rho, probs_dm(dm_probs(rho, petersen), petersen))
print("petersen gram:\n %s" % np.round(povm_gram(petersen, normalized=False), decimals=3))
print("quantumness: %f" % quantumness(petersen))

petersen gram:
 [[0.16  0.071 0.004 0.004 0.071 0.071 0.004 0.004 0.004 0.004]
 [0.071 0.16  0.071 0.004 0.004 0.004 0.071 0.004 0.004 0.004]
 [0.004 0.071 0.16  0.071 0.004 0.004 0.004 0.071 0.004 0.004]
 [0.004 0.004 0.071 0.16  0.071 0.004 0.004 0.004 0.071 0.004]
 [0.071 0.004 0.004 0.071 0.16  0.004 0.004 0.004 0.004 0.071]
 [0.071 0.004 0.004 0.004 0.004 0.16  0.004 0.071 0.071 0.004]
 [0.004 0.071 0.004 0.004 0.004 0.004 0.16  0.004 0.071 0.071]
 [0.004 0.004 0.071 0.004 0.004 0.071 0.004 0.16  0.004 0.071]
 [0.004 0.004 0.004 0.071 0.004 0.071 0.071 0.004 0.16  0.004]
 [0.004 0.004 0.004 0.004 0.071 0.004 0.071 0.071 0.004 0.16 ]]
quantumness: 34.047026


In $d=3$, there's a real SIC based on the icosahedron!

In [ ]:
#export 
def circular_shifts(v):
    shifts = [v]
    for i in range(len(v)-1):
        u = shifts[-1][:]
        u.insert(0, u.pop()) 
        shifts.append(u)
    return shifts

def icosahedron_vertices():
    phi = (1+np.sqrt(5))/2
    return [np.array(v) for v in 
               circular_shifts([0, 1, phi]) + \
               circular_shifts([0, -1, -phi]) + \
               circular_shifts([0, 1, -phi]) + \
               circular_shifts([0, -1, phi])]

def icosahedron_povm():
    vertices = icosahedron_vertices()
    keep = []
    for i, a in enumerate(vertices):
        for j, b in enumerate(vertices):
            if i != j and np.allclose(a, -b) and j not in keep:
                keep.append(i)
    vertices = [qt.Qobj(e).unit() for i, e in enumerate(vertices) if i in keep]
    return [(1/2)*v*v.dag() for v in vertices]

icosahedron = icosahedron_povm()
assert np.allclose(sum(icosahedron), qt.identity(3))

rho = real_rand_dm(3)
assert np.allclose(rho, probs_dm(dm_probs(rho, icosahedron), icosahedron))
print("icosahedron gram:\n %s" % np.round(povm_gram(icosahedron, normalized=False), decimals=3))
print("quantumness: %f" % quantumness(icosahedron))

icosahedron gram:
 [[0.25 0.05 0.05 0.05 0.05 0.05]
 [0.05 0.25 0.05 0.05 0.05 0.05]
 [0.05 0.05 0.25 0.05 0.05 0.05]
 [0.05 0.05 0.05 0.25 0.05 0.05]
 [0.05 0.05 0.05 0.05 0.25 0.05]
 [0.05 0.05 0.05 0.05 0.05 0.25]]
quantumness: 3.354102
